## Creating a proper Keras dataset

### Checking the path

In [ ]:
import os
os.getcwd()

### Importing python libraries

In [2]:
import os
import shutil
import json
import cv2
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

2023-07-25 18:42:18.822738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 18:42:21.142580: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Creating Keras folders and subfolders

In [9]:
import os
import shutil
import json
import cv2
from matplotlib import pyplot as plt

original_dataset_dir = "dataset/corn_kenel_counting_dataset/corn_coco"

# Opening JSON file
f_train = open(f'{original_dataset_dir}/corn_kernel_train.json')
f_test = open(f'{original_dataset_dir}/corn_kernel_test.json')

train_classes_mapper = json.load(f_train)
test_classes_mapper = json.load(f_test)

# def creating_folders(original_dataset_dir, classes_mapper):
    
#     parent = os.getcwd()
    
#     base_dir = f"data"
#     os.mkdir(base_dir)
    
#     train_dir = os.path.join(base_dir, "training")
#     os.mkdir(train_dir)
#     test_dir = os.path.join(base_dir, "testing")
#     os.mkdir(test_dir)
#     valid_dir = os.path.join(base_dir, "validation")
#     os.mkdir(valid_dir)
    
#     for c in range(len(classes_mapper["categories"])):

#         new_dir = os.path.join(train_dir, f"{classes_mapper['categories'][c]['name']}")
#         os.mkdir(new_dir)
        
#     for c in range(len(classes_mapper["categories"])):

#         new_dir = os.path.join(test_dir, f"{classes_mapper['categories'][c]['name']}")
#         os.mkdir(new_dir)
        
#     for c in range(len(classes_mapper["categories"])):

#         new_dir = os.path.join(valid_dir, f"{classes_mapper['categories'][c]['name']}")
#         os.mkdir(new_dir)

def creating_folders( classes_mapper):
    
    parent = os.getcwd()
    
    base_dir = f"input"
    os.mkdir(base_dir)

    
    for c in range(len(classes_mapper["categories"])):

        new_dir = os.path.join(base_dir, f"{classes_mapper['categories'][c]['name']}")
        os.mkdir(new_dir)


creating_folders(train_classes_mapper)

### Organizing images in the input folder

In [ ]:

    
def creating_image_dataset(classes_mapper, base_dir, original_dataset_dir, index=0):
    
    ear_classes = {"26560": "Kernel", 
           "26561": "Diseased Kernel", 
           "26562": "Incomplete Kernel", 
           "26563": "Baren Tip", 
           "33761": "Kernel Area"}

    # Iterate through the files
    for i in range(len(classes_mapper["images"])):
        
        for a in range(len(classes_mapper["annotations"])):
 
            if classes_mapper["images"][i]["id"] == classes_mapper["annotations"][a]["image_id"]:

                image = cv2.imread(f"{original_dataset_dir}/{classes_mapper['images'][i]['file_name']}")
            
                x, y, width, height = list(map(int, classes_mapper["annotations"][a]["bbox"]))
                
                cropped_image = image[y: (height + y), x: (width + x)]

                id_annotation = int(classes_mapper["annotations"][a]['id']) + index
                
                label = ear_classes[str(classes_mapper["annotations"][a]["category_id"])]
                print(label, id_annotation)

                if 1 <= id_annotation < 10:
                    cv2.imwrite(f"{base_dir}/{label}/0000{id_annotation}.jpg", cropped_image)

                elif 10 <= id_annotation < 100:
                    cv2.imwrite(f"{base_dir}/{label}/000{id_annotation}.jpg", cropped_image)

                elif 100 <= id_annotation < 1000:
                    cv2.imwrite(f"{base_dir}/{label}/00{id_annotation}.jpg", cropped_image)

                elif 1000 <= id_annotation < 10000:
                    cv2.imwrite(f"{base_dir}/{label}/0{id_annotation}.jpg", cropped_image)

                else:
                    cv2.imwrite(f"{base_dir}/{label}/{id_annotation}.jpg", cropped_image)
                    
    print("Finished")
    
creating_image_dataset(
    test_classes_mapper, 
    "input", 
    original_dataset_dir)

creating_image_dataset(
    train_classes_mapper, 
    "input", 
    original_dataset_dir,
    index = 16010)


### Spliting the dataset

In [1]:
import splitfolders

splitfolders.ratio("input", output="data",
    seed=42, ratio=(.7, .1, .2), group_prefix=None, move=False) 


### Preprocessing the input data

In [ ]:
# Preprocessing the input data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(224, 224),
    batch_size=20,
    class_mode='categorical',
    seed=42
)

validation_generator = test_datagen.flow_from_directory(
    "data/val",
    target_size=(224, 224),
    batch_size=20,
    class_mode='categorical',
    seed=42
)

## Keras Models

### MobileNetV2

In [20]:
# Downloading the model
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator

model = MobileNetV2(weights='imagenet')

Found 50655 images belonging to 5 classes.
Found 7235 images belonging to 5 classes.


### Customizing the model

In [35]:

from keras import Model
from keras.layers import Dense
import keras

corn_output = Dense(5, activation='softmax')
corn_output = corn_output(model.layers[-2].output)
corn_input = model.input
corn_model = Model(inputs=corn_input, outputs=corn_output)

for layer in corn_model.layers[:-1]:
    layer.trainable

corn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=10,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    tf.keras.callbacks.CSVLogger(
        "Log", 
        separator=",", 
        append=False
    )
]

In [36]:
corn_model.fit(
    train_generator,
    epochs=100,
    verbose=2,
    validation_data=validation_generator,
    callbacks=callbacks_list
)

model.save('corn_classification.h5')

Epoch 1/100


2023-07-25 20:00:03.606429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-07-25 20:00:13.785362: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-25 20:00:19.727472: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 79.09MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-25 20:00:19.829024: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 98.18MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/usr/lib/python3/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_onc